In [1]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

## Reading in the data and preparing DataFrames

#### MID - Exceptional case with only 1 MID channel

In [2]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
# #bring the time data into a new df:
time = pd.DataFrame(mid.iloc[:,0]) #- get time column
masses = [2]

#add elapsed time columns
def elapsed_time(df):
    counter = len(df.columns)
    for idx, col in enumerate(df):
        timestamp = pd.to_datetime(df[col], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp - timestamp[0]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[idx]) 
        df.insert(counter, colname, runtime)
        counter += 1
    return df
print(elapsed_time(time))
#print(time)
#print(elapsed_time)
time_sliced = time.iloc[:,1].fillna(0) #select the time columns
print(time_sliced)

#convert elapsed times in time-dataframe back to a timestamp
def h2_time(df1,df2):
    startdate = pd.to_datetime(df1.iloc[0,0],format="%Y/%m/%d %H:%M:%S.%f")
    print(startdate)
    dic= {}
    for idx, mass in enumerate(masses):
        key = "Timestamp, mass {}".format(mass)
        content = df2.dropna().values
        print(content)
        for row in content:
            dic.setdefault(key,[]).append((startdate+dt.timedelta(milliseconds=float(row)*1000)))
    new_df = pd.DataFrame.from_dict(dic)
    return(new_df)

new_df = h2_time(time,time_sliced)
new_df.columns

                 mass 2.0_time  Elapsed time, mass 2
0      2023/05/09 16:05:17.209                 0.000
1      2023/05/09 16:05:17.480                 0.271
2      2023/05/09 16:05:20.752                 3.543
3      2023/05/09 16:05:21.023                 3.814
4      2023/05/09 16:05:21.294                 4.085
...                        ...                   ...
12812  2023/05/09 17:03:15.764              3478.555
12813  2023/05/09 17:03:16.035              3478.826
12814  2023/05/09 17:03:16.306              3479.097
12815  2023/05/09 17:03:16.577              3479.368
12816  2023/05/09 17:03:16.848              3479.639

[12817 rows x 2 columns]
0           0.000
1           0.271
2           3.543
3           3.814
4           4.085
           ...   
12812    3478.555
12813    3478.826
12814    3479.097
12815    3479.368
12816    3479.639
Name: Elapsed time, mass 2, Length: 12817, dtype: float64
2023-05-09 16:05:17.209000
[0.000000e+00 2.710000e-01 3.543000e+00 ... 3.479097e+0

Index(['Timestamp, mass 2'], dtype='object')

#### Vaclogger

In [3]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,100mbar inj,Barion_2,Barion_1,DUAL experiment,DUAL insulation,1mbar inj,helium,T-platinum,T-CERNOX,V_heater,DUAL inj line,I_emission,I_grid
0,NaN,2023-05-09 15:58:45,0.0,-0.017646,1.970000e-09,8.180000e-10,4.996000e-09,6.268000e-07,-0.000791,0.0,28.283,158.136,0.000008,0.000006,NaN,NaN
1,NaN,2023-05-09 15:58:54,9.0,-0.017670,1.990000e-09,8.180000e-10,4.996000e-09,6.283000e-07,-0.000794,0.0,28.283,158.024,0.000008,0.000006,NaN,NaN
2,NaN,2023-05-09 15:59:02,17.0,-0.017375,1.980000e-09,8.180000e-10,4.996000e-09,6.261000e-07,-0.000789,0.0,28.283,158.024,0.000007,0.000006,NaN,NaN
3,NaN,2023-05-09 15:59:11,26.0,-0.017005,1.980000e-09,8.150000e-10,4.996000e-09,6.283000e-07,-0.000783,0.0,28.395,158.136,0.000007,0.000006,NaN,NaN
4,Temp around 100K,2023-05-09 15:59:19,34.0,-0.017595,1.990000e-09,8.180000e-10,4.996000e-09,6.283000e-07,-0.000796,0.0,28.283,158.024,0.000008,0.000006,NaN,NaN


In [4]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")

#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["I_em"]=[element * 1e-6 for element in hv["IMon2"]]

# hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["I_em"]=abs(hv["I_em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding a datetime column
datetime = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
datetime = datetime.dt.strftime('%d-%m-%Y %H:%M:%S')
datetime = pd.to_datetime(datetime, format='%d-%m-%Y %H:%M:%S')
hv.insert(0,"datetime",datetime)
    
#making the column into a datetime object
hv_timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(hv_timestamps-hv_timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)

hv.head()



,datetime,Date,Elapsed time,Time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,...,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,Unnamed: 19,hv_grid,I_em
0,2023-05-09 16:05:26,09/05/2023,0.000,16:05:26.668,0.0,0.04,0.06,0.08,0.0,0.0,...,0.0092,0.0081,0.011,0.0069,0.0069,-0.0115,NaN,NaN,1.400000e-08,9.300000e-09
1,2023-05-09 16:05:27,09/05/2023,1.183,16:05:27.851,0.0,0.04,0.06,0.08,0.0,0.0,...,0.0092,0.0081,0.011,0.0069,0.0069,-0.0115,NaN,NaN,1.400000e-08,9.300000e-09
2,2023-05-09 16:05:29,09/05/2023,2.365,16:05:29.033,0.0,0.04,0.06,0.08,0.0,0.0,...,0.0092,0.0081,0.011,0.0069,0.0069,-0.0115,NaN,NaN,1.400000e-08,9.300000e-09
3,2023-05-09 16:05:30,09/05/2023,3.544,16:05:30.212,0.0,0.04,0.06,0.08,0.0,0.0,...,0.0092,0.0081,0.011,0.0069,0.0069,-0.0115,NaN,NaN,1.400000e-08,9.300000e-09
4,2023-05-09 16:05:31,09/05/2023,4.729,16:05:31.397,0.0,0.04,0.06,0.08,0.0,0.0,...,0.0088,0.0081,0.011,0.0074,0.0069,-0.0115,NaN,NaN,1.400000e-08,9.300000e-09


## Data processing

#### Comments

In [5]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan 'Temp around 100K' 'end of esd measurement']


In [6]:
#print hv comments
print(pd.unique(hv["Comment"]))
hv_comm = hv.loc[:,["datetime","Comment"]]
hv_comm.dropna(inplace=True)
hv_comm.index

[nan 'grid transparent' 'grid repelling' 'grid transparent 2'
 'grid repelling 3' 'grid transparent 3' 'grid repelling 4'
 'grid transparent 4' 'grid repelling 5' 'grid transparent 5'
 'grid transparent 6']


Int64Index([ 497,  498,  499,  500,  797,  798,  799,  800, 1012, 1013, 1014,
            1015, 1370, 1371, 1372, 1373, 2018, 2019, 2020, 2021, 2134, 2135,
            2136, 2137, 2342, 2343, 2344, 2345, 2487, 2488, 2489, 2490, 2603,
            2604, 2605, 2606, 2727, 2728, 2729, 2730],
           dtype='int64')

#### CernOx temperature R-T conversion

In [7]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]

#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


## Plotting

In [8]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [9]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(12,6))

plt.plot(new_df["Timestamp, mass 2"],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')

plt.xlabel('Timestamp')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("S1 ESD @100K CU EGA halfway - MID scan")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(14))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
""" 
#write comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.95,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)

 """
plt.savefig(r'./graphs/MID.png')
plt.show()


In [10]:
#plotting HiVolta log
plt.figure(figsize=(12,6))

plt.plot(hv["datetime"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["datetime"],hv["I_em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Timestamp')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("S1 ESD @100K EGA halway - HiVolta log")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.8,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)
    
plt.savefig(r'./graphs/HiVolta.png')
plt.show()


In [11]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
#plt.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='DUAL experiment')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S1 ESD @100K CU, EGA halfway - Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./graphs/Pressures.png')
plt.show()

In [12]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S1 ESD @100K CU EGA halfway - Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp.png')
plt.show()